## EDA

- Permite ao cientista tomar decisões críticas sobre o que é interessante perseguir, e assim construindo uma hipótese a partir do relacionamento entre as variáveis.

### Passos

1. Limpeza
2. Definição de questões
3. Visualização de relacionamentos/resultados

In [1]:
import pandas as pd

In [2]:
ks = pd.read_csv('../../../datasets/ks-projects-201801.csv')
ks = ks.sample(30000, random_state=42).reset_index().drop('index', axis=1)  # ~10%

In [3]:
from utils import *
resumetable(ks)

Dataset Shape: (30000, 15)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,ID,int64,0,30000,1576537356,675907016,361890770,14.87
1,name,object,0,29983,Deko,"Westside BJ's: The Gluten-Free, Organic Food T...",Crepe Diem Food Truck,14.87
2,category,object,0,158,Hardware,Food Trucks,Food,6.26
3,main_category,object,0,15,Technology,Food,Food,3.57
4,currency,object,0,14,USD,USD,USD,1.28
5,deadline,object,0,2916,2015-10-24,2015-02-01,2014-01-17,11.23
6,goal,float64,0,1531,70000,250000,30000,6.55
7,launched,object,0,29997,2015-09-24 03:12:52,2015-01-02 20:55:07,2013-12-18 03:26:04,14.87
8,pledged,float64,0,10421,1888,1466,5723,10.72
9,state,object,0,6,failed,failed,failed,1.51


- Quais colunas não são úteis se desejamos predizer o sucesso (ou não) de um projeto?
  - De cara, sabemos que `ID` e `name` são desnecessárias.
  - `category` tem 158 valores únicos, então pode ser importante para classificação.
  - `main_category` com apenas 15 valores únicos talvez não seja útil sozinho, mas em combinação com outras features, como `category` pode importar mais.
    - Igualmente para os atributos `currency` ou `country`. Um ou outro porque pode ser que estejam correlacionados, logo adicionando uma redundância desnecessária.
  - `deadline`e `launched` são do formato `datetime`, podem ser úteis para indicar alguma tendência--por exemplo, melhor mês para iniciar/terminar o projeto.
     - Também podem ser úteis para derivar dados, como `campaign_time = deadline - launched`
  - `goal` e `pledged` estão na moeda indicada por `currency`, mas os atributos `usd_pledged_real` e `usd_goal_real` indicam a mesma coisa.
     - `usd pledged` indica o mesmo que `usd_pledged_real`: convesão de `pledged` para dólar, só que feito pelo time do kickstarter. 
     - `usd_pledged_real` (e `pledged`) é quanto o projeto conseguiu arrecadar. Para predição isso é inválido pois não se sabe seu valor à priori.
     - Sendo assim, podemos dropar `goal`, `pledged`, `usd pledged` e `usd_pledged_eal`.
       - Todavia, apesar de não sabermos o valor arrecadado à priori, pode ser útil para indicar que tipos de projetos arrecadam menos.
       - O mesmo vale para `backers`, que é quantos indivíduos ajudaram.
  - `state` é o resultado final, inicialmente dividido em 5 categorias: failed, canceled, successful, suspended, live, undefined.
    - Podemos dropar os projetos `live` e `undefined`, e juntar os `canceled` e `suspended` para `failed`.

## Recursos

- https://towardsdatascience.com/hitchhikers-guide-to-exploratory-data-analysis-6e8d896d3f7e